# Multiple-trait models for GWAS

In [ ]:
## You need to reinstall the required R packages each time you reinitiate an interactive sessino

#install.packages("qqman")
#install.packages("sommer")

In [ ]:
library("qqman")
library("sommer")
library("tidyverse")
library("data.table")

In [ ]:
trim <- function (x) gsub("^\\s+|\\s+$", "", x)
print("GWAS using the sommer package")

### Example data

Simulated fruit breeding data: [Covarrubias-Pazaran G (2016) Genome assisted prediction of quantitative traits using the R package sommer. PLoS ONE 11(6): doi:10.1371/journal.pone.0156744](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0156744)

In [ ]:
data(DT_cpdata)
DT <- na.omit(DT_cpdata) ## phenotypes
GT <- GT_cpdata[,sample(1:ncol(GT_cpdata), 500)] ## genotype data: subsample markers
MP <- MP_cpdata ## marker map file

We have a bunch of phenotypes (and covariables) on 290 fruit trees:

In [ ]:
print(nrow(DT)) ## n. of samples
head(DT)

In [ ]:
print(nrow(GT)) ## n. of samples
print(ncol(GT)) ## n. of markers
GT[1:8, 1:8]

In [ ]:
nrow(MP) ## n. of markers
head(MP)

Now we create a kinship matrix from the marker genotype data:

In [ ]:
#### create the variance-covariance matrix
A <- A.mat(GT) # additive relationship matrix
n <- nrow(DT) # to be used for degrees of freedom
k <- 1 # to be used for degrees of freedom (number of levels in fixed effects)

In [ ]:
heatmap(A)

In [ ]:
mix1 <- GWAS(color~1,
  random=~vs(id,Gu=A) + Rowf + Colf,
  rcov=~units,
  data=DT,
  M=GT,
  gTerm = "u:id",
  verbose = FALSE
)

In [ ]:
length(mix1$scores)
head(mix1$scores) ## (-log_(10)p) for each marker

In [ ]:
ms <- as.data.frame(mix1$scores)
ms$Locus <- rownames(ms)
MP2 <- merge(MP,ms,by="Locus",all.x = TRUE);
MP2 <- na.omit(MP2)
head(MP2) ## color is the -log(p-value)

In [ ]:
options(repr.plot.width=10, repr.plot.height=6)
manhattan(MP2, pch=20,cex=1.5, PVCN = "color")

In [ ]:
## function is deprecated and no longer maintained: it doesn't work any longer with more than one trait!!
mix2 <- GWAS(cbind(Yield, Firmness) ~ Rowf + Colf,
  random=~vs(id,Gu=A),
  rcov=~units,
  data=DT,
  M=GT,
  gTerm = "u:id",
  verbose = FALSE
)

---

## Our data

In [ ]:
genotype_file = "/content/rice_imputed.raw"
snp_map = "/content/rice_imputed.map"
phenotype_file = "/content/rice_phenotypes_multi.txt"
traits = "SL,SW"
covariates="population"

In [ ]:
print(paste("genotype file name:",genotype_file))
print(paste("SNP map:",snp_map))
print(paste("phenotype file name:",phenotype_file))
print(paste("trait:",traits))
covariates = if(exists(x = "covariates")) covariates else 1
print(paste("covariates:",covariates))

dataset = basename(genotype_file)

In [ ]:
## READING DATA
print("now reading in the data ...")
### genotypes
snp_matrix <- fread(genotype_file, header = TRUE)
print(paste(nrow(snp_matrix),"records read from the genotype file",sep=" "))
SNP_INFO <- fread(snp_map)
names(SNP_INFO) <- c("Chrom","snp","cM","Position")
SNP_INFO$cM <- NULL

X <- as.matrix(snp_matrix[,-c(1:6)])
colnames(X) <- gsub("\\_[A-Z]{1}$","",colnames(X))
rownames(X) <- snp_matrix$IID

In [ ]:
print(paste(nrow(SNP_INFO),"SNPs read from the map file",sep=" "))

if ((ncol(snp_matrix)-6) != nrow(SNP_INFO)) {

  stop("!! N. of SNPs in the map file not equal to the number of genotyped SNPs in the genotype file")

} else print("N. of SNPs in the map and genotype files is the same: this is correct!!")

In [ ]:
### phenotypes
phenotypes <- fread(phenotype_file)
# phenotypes <- phenotypes[,c(1,3)]
print(paste(nrow(phenotypes),"records read from the phenotype file",sep=" "))

phenotypes <- phenotypes[phenotypes$id %in% snp_matrix$IID,]
print(paste(nrow(phenotypes),"records read from the phenotype file after alignment with genotypes",sep=" "))

In [ ]:
## kinship matrix
print("Calculating the kinship matrix")
K <-A.mat(X)

vec <- colnames(K) %in% phenotypes$id
K <- K[vec,vec]

In [ ]:
heatmap(K)

Below, the long file format for the multiple-trai analysis (further down):

In [ ]:
pheno <- stackTrait(phenotypes, traits = c("SL","SW"))
head(pheno$long)

## GBLUP

### Single-trait

Let's first try a single trait GWAS on the phenotype SL (seed length)

In [ ]:
gblup1 <- mmes( SL ~ population,
              random=~vsm(ism(id),Gu=K),
              rcov=~units, nIters=10,
              data=phenotypes)

From GBLUP we can get also the **variance components** and the **estimated heritability** for the trait:

In [ ]:
summary(gblup1)$varcomp

In [ ]:
vpredict(gblup1, h2 ~ (V1) / ( V1+V2) )

No we need to retrieve **marker effects from genetic effects**: remember, we used a GBLUP model where the marker information was condensed in a (n x n) kinship matrix $\rightarrow$ one overall genetic value per individual (in this case, rice accessions)

Basically, we use the following identities:

$$
\hat{g} = \mathbf{X}\hat{\beta}
$$

$$
\hat{\beta} = \mathbf{X}'(\mathbf{X}\mathbf{X}')^{-1}\hat{g}
$$

$\mathbf{XX}'$ is a (n x m) * (m x n) = (n x n) matrix: this can be multiplied with the vector of individual genetic effects $\hat{g}$ to produce a new (n x 1) vector.

When we muliply this by $\mathbf{X}'$ (m x n), we obtain a vector of marker effects of size (m x 1)

The (n x n) $\mathbf{XX}'$ matrix is a **kinship matrix** (cross-product of the SNP matrix and its transposed, rescaled by allele frequencies):

In [ ]:
## getting the X'(XX')^-1 matrix
Kinv <- solve(K + diag(1e-6, ncol(K), ncol(K))) ## inverse of K
XKinv <- t(X)%*%Kinv # t(X) %*% K^-1

In [ ]:
dim(XKinv)

From GBLUP, we get the individual genetic effects:

In [ ]:
## individual genetic effects
g <- gblup1$uList$`vsm(ism(id), Gu = K)`

In [ ]:
## 140: one per rice accession
dim(g)

In [ ]:
# (m, n) x (n)
a.from.g <- XKinv %*% g

In [ ]:
dim(a.from.g)

In [ ]:
head(a.from.g)

Now we have the marker effects: to calculate their p-values, we need to calculate first the standard error of these estimated marker effects.

We'll need the genetic variance and the individual coefficients:

In [ ]:
## end/start of random effects in results: intercept, population (fixed effects: covariates), then 140 individual genotypes
gblup1$partitions

In [ ]:
start = gblup1$partitions[[1]][1]
end = gblup1$partitions[[1]][2]

In [ ]:
## Ci: inverse of the coefficient matrix
## from the MME, e.g.:
## | X'X X'Z                |
## | Z'X Z'Z + G^-1 + alpha |
dim(gblup1$Ci) ## mu, population, 140 individuals

In [ ]:
## the variance components
gblup1$theta

In [ ]:
head(g)

In [ ]:
## Ci is the inverse of the coefficient matrix
var.g <- kronecker(K, gblup1$theta[[1]]) - gblup1$Ci[start:end,start:end]

In [ ]:
dim(var.g)

In [ ]:
## (m, n) * (n,n) * (n,n) * (n,n) * (n,m)
var.a.from.g <- t(X) %*% Kinv %*% (var.g) %*% t(Kinv) %*% X

In [ ]:
## from (n x n) to (m x m) variance matrix: from each individual to each marker
dim(var.a.from.g)

In [ ]:
## one per SNP
se.a.from.g <- sqrt(diag(var.a.from.g))

With the standard error of the estimate, we can now calculate the `t statistic` value for each marker:

In [ ]:
## one per SNP
t.stat.from.g <- a.from.g/se.a.from.g # t-statistic

In [ ]:
n <- nrow(phenotypes) # to be used for degrees of freedom
k <- 1 # to be used for degrees of freedom (number of levels in fixed effects)

In [ ]:
pvalGBLUP <- dt(t.stat.from.g, df=n-k-1) # pvalues

In [ ]:
summary(pvalGBLUP)

In [ ]:
snp_names = rownames(a.from.g)
snp_effects = as.data.frame(a.from.g)
snp_effects$snp = snp_names
names(snp_effects)[1] <- "effect"
snp_effects$pval = pvalGBLUP
head(snp_effects)

In [ ]:
temp <- SNP_INFO |> inner_join(snp_effects, by = "snp")
temp <- temp |> mutate(log_pval = -log10(pval)) |> select(-effect, -pval)
head(temp)

In [ ]:
options(repr.plot.width=10, repr.plot.height=6)
manhattan(temp, pch=20,cex=1.5, PVCN = "log_pval")

### Multiple-trait

In [ ]:
gblup_multi <- mmes( valueS ~ trait + population, # henderson=TRUE,
              random=~ vsm(usm(trait), ism(id), Gu=K),
              rcov=~ vsm(dsm(trait), ism(units)),
              data=pheno$long)

In [ ]:
cov2cor(gblup_multi$theta[[1]])

In [ ]:
summary(gblup_multi)$varcomp

In [ ]:
## getting the X'(XX')^-1 matrix
Kinv <- solve(K + diag(1e-6, ncol(K), ncol(K)))
XKinv <- t(X)%*%Kinv

In [ ]:
## individual genetic effects
g <- gblup_multi$uList$`vsm(usm(trait), ism(id), Gu = K)`

We now have **two sets of genetic values**: one per trait!

In [ ]:
head(g)

In [ ]:
a.from.g <- XKinv %*% g

This brings to **two sets of marker effects**:

In [ ]:
dim(a.from.g)

#### Trait one: SL (seed length)

In [ ]:
## first trait
start_1 = gblup_multi$partitions[[1]][1]
end_1 = gblup_multi$partitions[[1]][3]

In [ ]:
## the variance components: genetic variance and covariance
gblup_multi$theta[[1]]

In [ ]:
## Ci is the inverse of the coefficient matrix
var.g <- kronecker(K, gblup_multi$theta[[1]][1]) - gblup_multi$Ci[start_1:end_1,start_1:end_1]

In [ ]:
## t statistic
var.a.from.g <- t(X) %*% Kinv %*% (var.g) %*% t(Kinv) %*% X ## variance of marker effects
se.a.from.g <- sqrt(diag(var.a.from.g))  ## standard error of the estimates
t.stat.from.g <- a.from.g[,1]/se.a.from.g # t-statistic

In [ ]:
n <- nrow(phenotypes) # to be used for degrees of freedom
k <- 1 # to be used for degrees of freedom (number of levels in fixed effects)
pval_1 <- dt(t.stat.from.g, df=n-k-1) # pvalues

In [ ]:
pval_1

#### Trait two: SW (seed width)

In [ ]:
## second trait
start_2 = gblup_multi$partitions[[1]][2]
end_2 = gblup_multi$partitions[[1]][4]

## choose the corresponding genetic variance; Ci is the inverse of the coefficient matrix
var.g <- kronecker(K, gblup_multi$theta[[1]][4]) - gblup_multi$Ci[start_2:end_2,start_2:end_2]

In [ ]:
## t statistic
var.a.from.g <- t(X) %*% Kinv %*% (var.g) %*% t(Kinv) %*% X ## variance of marker effects
se.a.from.g <- sqrt(diag(var.a.from.g))  ## standard error of the estimates
t.stat.from.g <- a.from.g/se.a.from.g # t-statistic

In [ ]:
n <- nrow(phenotypes) # to be used for degrees of freedom
k <- 1 # to be used for degrees of freedom (number of levels in fixed effects)
pval_2 <- dt(t.stat.from.g, df=n-k-1) # pvalues